In [4]:
⎕IO←0
⎕PW←12345
]box on

Was OFF


In [1]:
Depths←{ ⍝ find the depths of each node in a parent vector
    ⍝ ←: a vector of the depths of each node in the input
    p←⍵    ⍝ input parent vector
    depths←(≢p)⍴0
    StepUp←{ ⍝ step up the tree and increment depths
        q←p[⍵]
        depths+←⍵≠q
        q
    }
    _←StepUp⍣≡⍳≢p
    depths
}

_PrettyPrint_←{ ⍝ renders a tree given labels, box drawing characters, and padding
    ⍝ ←: vector of character matrices, each a labelled rendering of a tree in the forest given by the input parent vector
    labels    ←⍺     ⍝ vector of character matrices giving the labels for each node
    connectors←⍺⍺    ⍝ box drawing characters to render the tree, e.g: '─┌┬┐│┴├┼┤│' (normal, and upstruck)
    spaces    ←⍵⍵    ⍝ number of spaces to pad with between sub-trees
    p         ←⍵     ⍝ parent vector
    d←Depths p
    maxDepth←⌈/d
    results←labels         ⍝ result of rendering each sub-tree, seeded with labels
    maxDepth=0: results    ⍝ avoid the each running on the prototype
    DoFamily←{ ⍝ render and record a sub-tree
        ⍝ ⍺: parent node
        ⍝ ⍵: rendered results of children
        widths←(1⊃⍴)¨⍵                                                                           ⍝ widths of each rendered child
        width←spaces-⍨+/spaces+widths                                                            ⍝ eventual width of the rendered tree       wwwwwww
        centres←(+\0,¯1↓spaces+widths)+¯1+⌈2÷⍨widths                                             ⍝ centres of each rendered sub-tree         ∘ss∘ss∘
        result←width⍴' '                                                                         ⍝ header to be decorated                   '       '
        result[(⊢⊢⍤/⍨((⊃⌽centres)>⊢)∧(⊃centres)<⊢)⍳width]←connectors[0]                          ⍝ add horizontal bar                       ' ───── '
        result[   ⊃ centres]←connectors[1]                                                       ⍝ left end of bar                          '┌───── '
        result[   ⊃⌽centres]←connectors[3]                                                       ⍝ right end of bar                         '┌─────┐'
        result[¯1↓1↓centres]←connectors[2]                                                       ⍝ connectors to intermediate children      '┌──┬──┐'
        result[(1=≢centres)⍴⊃centres]←connectors[4]                                              ⍝ if there's only one child, just make it a lone upstrike
        centre←¯1+⌈2÷⍨width                                                                      ⍝ index of the centre of the rendered tree     ∘
        result[centre]←connectors[5 6 7 8 9][connectors[0 1 2 3 4]⍳result[centre]]               ⍝ connector to the parent                  '┌──┼──┐'
        result⍪←(-spaces)↓⍤1⊃,/,∘(spaces⍴' ')⍤1¨⍵↑¨⍨⌈/≢¨⍵                                        ⍝ pad labels, join under header
        parentResult←⍺⊃results                                                                   ⍝ label of the parent
        parentWidth←1⊃⍴parentResult                                                              ⍝ width of label of parent
        parentCentre←¯1+⌈2÷⍨parentWidth                                                          ⍝ centre of label of parent
        result      ←((centre-parentCentre)⌽parentWidth↑⍤1⊢)⍣(width<parentWidth)⊢result          ⍝ pad and recentre text so far if it's less wide
        parentResult←((parentCentre-centre)⌽      width↑⍤1⊢)⍣(width>parentWidth)⊢parentResult    ⍝ pad and recentre parent label if it's less wide
        result⍪⍨←parentResult                                                                    ⍝ add parent label
        results[⍺]←⊂result                                                                       ⍝ record result
        1
    }
    DoLayer←{ ⍝ render and record all nodes whose children have depth ⍵
        ⍝ ⍵: depth to handle nodes at
        i←⍸d=⍵    ⍝ nodes at this depth
        _←p[i]DoFamily⌸results[i]
        1
    }
    _←DoLayer¨⌽1+⍳maxDepth    ⍝ bottom up accumulation
    results/⍨p=⍳≢p            ⍝ return results at roots only
}

PPV←{⍺←'∘' ⋄   ((≢⍵)⍴⍉⍤⍪⍤⍕¨'∘'@(0=≢¨)⍺)('─┌┬┐│┴├┼┤│'_PrettyPrint_ 1)⍵}    ⍝ vertical
PPH←{⍺←'∘' ⋄ ⍉¨((≢⍵)⍴  ⍪⍤⍕¨'∘'@(0=≢¨)⍺)('│┌├└─┤┬┼┴─'_PrettyPrint_ 0)⍵}    ⍝ horizontal

---
```










```
---

# What can Vectorised Trees Do for You? - DYNA 2025

In [5]:
⍝      ┌─┐─────┐ ┌─┐─┐─┐
⍝      ↓ │     │ ↓ │ │ │
parent←0 0 1 1 0 4 5 5 5 0
⍝      ↑ ↑ │ │ ↑ │       │
⍝      │ └─┘─┘ └─┘       │
⍝      └─────────────────┘
labels←'abefcghijd'
labels PPV parent

┌───────────┐
│     a     │
│ ┌───┴┬───┐│
│ b    c   d│
│┌┴┐   │    │
│e f   g    │
│    ┌─┼─┐  │
│    h i j  │
└───────────┘


---
```










```
---

In [9]:
⍝ finding children of particular nodes
kids←parent∊labels⍳'bc'
kids PPV parent
labels[⍸kids]

┌───────────┐
│     0     │
│ ┌───┴┬───┐│
│ 0    0   0│
│┌┴┐   │    │
│1 1   1    │
│    ┌─┼─┐  │
│    0 0 0  │
└───────────┘


efg


---
```










```
---

In [10]:
⍝ finding the leaves
leaves←~(⍳≢parent)∊parent
leaves PPV parent
labels[⍸leaves]

┌───────────┐
│     0     │
│ ┌───┴┬───┐│
│ 0    0   1│
│┌┴┐   │    │
│1 1   0    │
│    ┌─┼─┐  │
│    1 1 1  │
└───────────┘


efhijd


---
```










```
---

In [11]:
⍝ trim nodes
i←labels⍳'c'
parent[i]←i
labels PPV parent

┌─────┬─────┐
│  a  │  c  │
│ ┌┴─┐│  │  │
│ b  d│  g  │
│┌┴┐  │┌─┼─┐│
│e f  │h i j│
└─────┴─────┘


---
```










```
---

In [15]:
⍝ you can have any kind of data associated
numbers←3 1 4 1 5 9 2 6 5 4
numbers PPV parent

depth←0 1 2 2 0 1 2 2 2 1
depth PPV parent

┌─────┬─────┐
│  3  │  5  │
│ ┌┴─┐│  │  │
│ 1  4│  9  │
│┌┴┐  │┌─┼─┐│
│4 1  │2 6 5│
└─────┴─────┘


┌─────┬─────┐
│  0  │  0  │
│ ┌┴─┐│  │  │
│ 1  1│  1  │
│┌┴┐  │┌─┼─┐│
│2 2  │2 2 2│
└─────┴─────┘


---
```










```
---

In [17]:
⍝ bottom up traversal
total←numbers
{
    i←⍸depth=⍵
    total[parent[i]]+←total[i] ⍝ the important line
}¨⌽⍳1+⌈/depth
total PPV parent

┌─────┬─────┐
│  26 │  54 │
│ ┌┴─┐│  │  │
│ 6  4│  22 │
│┌┴┐  │┌─┼─┐│
│4 1  │2 6 5│
└─────┴─────┘


---
```










```
---

In [21]:
json←⎕SH 'tree -J -s -a desktop'
(depth key value _)←↓⍉⎕JSON⎕OPT'Format' 'M'⊢∊json

⍝ get parent vector
parent←⍳≢depth
2{parent[⍵]←⍺[⍺⍸⍵]}/⊂⍤⊢⌸depth

⍝ discard report (the second element of the two-element array)
mask←1@1{parent[⍵]}@(≠∘1)⍣≡parent
parent←0@0⊢¯1+mask/parent
key  /⍨←mask
value/⍨←mask
depth/⍨←mask
depth-←1

⍝ bring fields out to their own columns
n←≢parent
keep  ←n⍴0
name  ←n⍴⊂''
size  ←n⍴0
isDir ←n⍴0
isFile←n⍴0

i←⍸'name'∘≡¨key ⋄ name  [parent[i]]←              value[i]
i←⍸'size'∘≡¨key ⋄ size  [parent[i]]←              value[i]
i←⍸'type'∘≡¨key ⋄ isDir [parent[i]]←'directory'∘≡¨value[i]
                  isFile[parent[i]]←     'file'∘≡¨value[i]

⍝ we don't use key, value or isDir after this point
⍝ (isDir ←→ ~isFile after this next bit)

⍝ delete superfluous field nodes and simplify the tree
mask←isDir∨isFile
i←⍸mask
parent[i]←parent[parent[i]]
parent/⍨←mask
parent-←(+\~mask)[parent]
name  /⍨←mask
size  /⍨←mask
isFile/⍨←mask
depth /⍨←mask
depth  ÷←2

    desktop/
    ├── drawer
    │   ├── document.md
    │   └── notebook.ipynb
    ├── empty-mug
    ├── my-wallet
    │   ├── card-slot
    │   │   └── id
    │   └── coin-pouch
    │       ├── cent
    │       ├── dime
    │       ├── nickel
    │       └── quarter
    ├── pen.cil
    ├── plant-pot
    │   └── aloe.vera
    ├── .secret-compartment
    │   ├── chocolate.bar
    │   └── ufo.evidence
    └── todo.txt

In [22]:
⍝ names of files larger than ⍵
name/⍨isFile∧size>100

┌───────────┬──────────────┬─────────┬─────────────┐
│document.md│notebook.ipynb│aloe.vera│chocolate.bar│
└───────────┴──────────────┴─────────┴─────────────┘


---
```










```
---

In [26]:
⍝ usage of immediate subdirectories
usage←size
{
    i←⍸depth=⍵
    usage[parent[i]]+←usage[i]
}¨⌽1+⍳⌈/depth
i←1↓⍸parent=0
n←usage[i]
names←name[i]
((⍕⌈/≢¨names),'A1,X2,I3,X2,F5.2,⊂%⊃') ⎕FMT (↑names) n (100×n÷+/n)

drawer               314  17.27%
empty-mug              0   0.00%
my-wallet            145   7.98%
pen.cil               77   4.24%
plant-pot            792  43.56%
.secret-compartment  395  21.73%
todo.txt              95   5.23%


---
```










```
---

In [28]:
hidden←⍸'.'=⊃¨name
hidden←0≠{parent[⍵]}@{~⍵∊hidden}⍣≡⍳≢parent
mask←~hidden
parent/⍨←mask
parent-←(+\hidden)[parent]
name/⍨←mask
name PPH parent

┌────────────────────────────────────┐
│       ┌drawer┬document.md          │
│       │      └notebook.ipynb       │
│       ├empty-mug                   │
│       │         ┌card-slot─id      │
│       │         │          ┌cent   │
│desktop┼my-wallet┴coin-pouch┼dime   │
│       │                    ├nickel │
│       │                    └quarter│
│       ├pen.cil                     │
│       ├plant-pot─aloe.vera         │
│       └todo.txt                    │
└────────────────────────────────────┘


---
```










```
---